In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession 

sc = SparkContext()
spark = SparkSession.builder.appName("HW4-1").getOrCreate()

In [2]:
spotify = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/training_set/")

In [4]:
#feature = spark.read.format("csv").option("header", "true").load("s3://502-mountain-dew-spotify/track_features")

## Merge Data

In [3]:
df = spotify.join(feature, spotify.track_id_clean == feature.track_id , "inner")
df.printSchema()

NameError: name 'feature' is not defined

In [6]:
df.count()

2990609

In [7]:
df.na.drop().count()

2990609

In [10]:
df.cache()

DataFrame[session_id: string, session_position: string, session_length: string, track_id_clean: string, skip_1: string, skip_2: string, skip_3: string, not_skipped: string, context_switch: string, no_pause_before_play: string, short_pause_before_play: string, long_pause_before_play: string, hist_user_behavior_n_seekfwd: string, hist_user_behavior_n_seekback: string, hist_user_behavior_is_shuffle: string, hour_of_day: string, date: string, premium: string, context_type: string, hist_user_behavior_reason_start: string, hist_user_behavior_reason_end: string, track_id: string, duration: string, release_year: string, us_popularity_estimate: string, acousticness: string, beat_strength: string, bounciness: string, danceability: string, dyn_range_mean: string, energy: string, flatness: string, instrumentalness: string, key: string, liveness: string, loudness: string, mechanism: string, mode: string, organism: string, speechiness: string, tempo: string, time_signature: string, valence: string, 

## Change Data Type

In [4]:
spotify.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- session_position: string (nullable = true)
 |-- session_length: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: string (nullable = true)
 |-- skip_2: string (nullable = true)
 |-- skip_3: string (nullable = true)
 |-- not_skipped: string (nullable = true)
 |-- context_switch: string (nullable = true)
 |-- no_pause_before_play: string (nullable = true)
 |-- short_pause_before_play: string (nullable = true)
 |-- long_pause_before_play: string (nullable = true)
 |-- hist_user_behavior_n_seekfwd: string (nullable = true)
 |-- hist_user_behavior_n_seekback: string (nullable = true)
 |-- hist_user_behavior_is_shuffle: string (nullable = true)
 |-- hour_of_day: string (nullable = true)
 |-- date: string (nullable = true)
 |-- premium: string (nullable = true)
 |-- context_type: string (nullable = true)
 |-- hist_user_behavior_reason_start: string (nullable = true)
 |-- hist_user_behavior_reason_end: string

In [5]:
columns_to_drop = ["hist_user_behavior_reason_start", "hist_user_behavior_reason_end", "context_type", "date"
                  ,"hour_of_day", "hist_user_behavior_is_shuffle", "hist_user_behavior_n_seekfwd", "session_length","session_position"]
df_drop = spotify.drop(*columns_to_drop)
df_drop.show(1)

+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|no_pause_before_play|short_pause_before_play|long_pause_before_play|hist_user_behavior_n_seekback|premium|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|             0|                   0|                      0|                     0|                            0|   true|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
only showi

In [6]:
from pyspark.sql.types import *

for c in ["skip_1", "skip_2", "skip_3", "not_skipped", "context_switch", "no_pause_before_play",
         "short_pause_before_play", "long_pause_before_play", "premium"]:
    df_drop = df_drop.withColumn(c, df_drop[c].cast(BooleanType()))
for c in ["hist_user_behavior_n_seekback"]:
    df_drop = df_drop.withColumn(c, df_drop[c].cast(IntegerType()))  
    
df_drop.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: boolean (nullable = true)
 |-- skip_2: boolean (nullable = true)
 |-- skip_3: boolean (nullable = true)
 |-- not_skipped: boolean (nullable = true)
 |-- context_switch: boolean (nullable = true)
 |-- no_pause_before_play: boolean (nullable = true)
 |-- short_pause_before_play: boolean (nullable = true)
 |-- long_pause_before_play: boolean (nullable = true)
 |-- hist_user_behavior_n_seekback: integer (nullable = true)
 |-- premium: boolean (nullable = true)



## Rating

In [7]:
df_drop.show(1)

+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|no_pause_before_play|short_pause_before_play|long_pause_before_play|hist_user_behavior_n_seekback|premium|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|         false|               false|                  false|                 false|                            0|   true|
+--------------------+--------------------+------+------+------+-----------+--------------+--------------------+-----------------------+----------------------+-----------------------------+-------+
only showi

In [8]:
from pyspark.sql.functions import col

def pause_udf(short_pause, long_pause):
    if short_pause or long_pause:
        return 0
    else:
        return 1
    
pause = spark.udf.register("pause", pause_udf)

df1 = df_drop.select("session_id", "track_id_clean", "skip_1", "skip_2", "skip_3",
                     "not_skipped", "context_switch", "hist_user_behavior_n_seekback",
                     "premium", pause(col("short_pause_before_play"), col("long_pause_before_play")).alias("Not_Pause"))
df1.show(1)

#df_drop1 = df_drop.join(df1, ["session_id"] , "left")
df1 = df1.withColumn("Not_Pause", df1["Not_Pause"].cast(IntegerType()))
df1.printSchema()

+--------------------+--------------------+------+------+------+-----------+--------------+-----------------------------+-------+---------+
|          session_id|      track_id_clean|skip_1|skip_2|skip_3|not_skipped|context_switch|hist_user_behavior_n_seekback|premium|Not_Pause|
+--------------------+--------------------+------+------+------+-----------+--------------+-----------------------------+-------+---------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...| false| false| false|       true|         false|                            0|   true|        1|
+--------------------+--------------------+------+------+------+-----------+--------------+-----------------------------+-------+---------+
only showing top 1 row

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- skip_1: boolean (nullable = true)
 |-- skip_2: boolean (nullable = true)
 |-- skip_3: boolean (nullable = true)
 |-- not_skipped: boolean (nullable = true)
 |-- context_switch: b

In [9]:
def skip_rating_udf(premium, skip_1, skip_2, skip_3, not_skip, nopause):
    if premium:
        if skip_1:
            skip1 = 1 - (1 - nopause)
        else:
            skip1 = 0
        if skip_2:
            skip2 = 3
        else:
            skip2 = 0
        if skip_3:
            skip3 = 5
        else:
            skip3 = 0
        if not_skip:
            notskip = 6 + (2 - 2 * nopause)
        else:
            notskip = 0
        return max(skip1, skip2, skip3, notskip)
    else:
        if skip_1:
            skip1 = 0
        else:
            skip1 = 0
        if skip_2:
            skip2 = 1
        else:
            skip2 = 0
        if skip_3:
            skip3 = 2
        else:
            skip3 = 0
        if not_skip:
            notskip = 5 + (1 - 1 * nopause)
        else:
            notskip = 0
        return max(skip1, skip2, skip3, notskip)
    
skip_rating = spark.udf.register("skip_rating", skip_rating_udf)

df2 = df1.select("session_id","context_switch","track_id_clean",
                 "hist_user_behavior_n_seekback",skip_rating(col("premium"), col("skip_1"), 
                                                             col("skip_2"), col("skip_3"), col("not_skipped"), 
                                                             col("Not_Pause")).alias("Skip_Rating"))
df2.show(5)

#df_drop2 = df_drop1.join(df2, ["session_id"] , "left")
df2 = df2.withColumn("Skip_Rating", df2["Skip_Rating"].cast(IntegerType()))

+--------------------+--------------+--------------------+-----------------------------+-----------+
|          session_id|context_switch|      track_id_clean|hist_user_behavior_n_seekback|Skip_Rating|
+--------------------+--------------+--------------------+-----------------------------+-----------+
|0_00006f66-33e5-4...|         false|t_0479f24c-27d2-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_9099cd7b-c238-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_fc5df5ba-5396-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_23cff8d6-d874-4...|                            0|          6|
|0_00006f66-33e5-4...|         false|t_64f3743c-f624-4...|                            0|          6|
+--------------------+--------------+--------------------+-----------------------------+-----------+
only showing top 5 rows



In [10]:
def final_rating_udf(Skip_Rating, context_switch, seekback):
    if context_switch:
        context = 1
    else:
        context = 0
    inside = 1 - (Skip_Rating + 2*context)/10
    seekback = seekback + 1
    return (1 - pow(inside, (pow(seekback, 1/3)))) * 10

final_rating = spark.udf.register("final_rating", final_rating_udf)

df3 = df2.select("session_id", "track_id_clean",
                 final_rating(col("Skip_Rating"), col("context_switch"),
                              col("hist_user_behavior_n_seekback")).alias("Final_Rating"))
df3.show(5)

#df_drop3 = df_drop2.join(df3, ["session_id"] , "left")
df3 = df3.withColumn("Final_Rating", df3["Final_Rating"].cast(IntegerType()))

+--------------------+--------------------+------------+
|          session_id|      track_id_clean|Final_Rating|
+--------------------+--------------------+------------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...|         6.0|
|0_00006f66-33e5-4...|t_9099cd7b-c238-4...|         6.0|
|0_00006f66-33e5-4...|t_fc5df5ba-5396-4...|         6.0|
|0_00006f66-33e5-4...|t_23cff8d6-d874-4...|         6.0|
|0_00006f66-33e5-4...|t_64f3743c-f624-4...|         6.0|
+--------------------+--------------------+------------+
only showing top 5 rows



In [11]:
def adjust_udf(final):
    return final + 1

adjust = spark.udf.register("adjust", adjust_udf)

df4 = df3.select("session_id", "track_id_clean",
                 adjust(col("Final_Rating")).alias("Final_Rating_Adjust"))
df4.show(5)

#df_drop3 = df_drop2.join(df3, ["session_id"] , "left")
df4 = df4.withColumn("Final_Rating_Adjust", df4["Final_Rating_Adjust"].cast(IntegerType()))

+--------------------+--------------------+-------------------+
|          session_id|      track_id_clean|Final_Rating_Adjust|
+--------------------+--------------------+-------------------+
|0_00006f66-33e5-4...|t_0479f24c-27d2-4...|                  7|
|0_00006f66-33e5-4...|t_9099cd7b-c238-4...|                  7|
|0_00006f66-33e5-4...|t_fc5df5ba-5396-4...|                  7|
|0_00006f66-33e5-4...|t_23cff8d6-d874-4...|                  7|
|0_00006f66-33e5-4...|t_64f3743c-f624-4...|                  7|
+--------------------+--------------------+-------------------+
only showing top 5 rows



In [12]:
df4.write.format("csv").option("header","true").mode("Overwrite").save("s3://olihw4/spotify_full_version")

## Distribution 

In [13]:
df4.describe("Final_Rating_Adjust").show()

+-------+-------------------+
|summary|Final_Rating_Adjust|
+-------+-------------------+
|  count|         2072002577|
|   mean|  5.916154485072342|
| stddev| 1.7028280992338205|
|    min|                  1|
|    max|                 11|
+-------+-------------------+



In [15]:
from pyspark.sql.functions import count
df4.groupBy("Final_Rating_Adjust").agg(count('session_id')).show()

+-------------------+-----------------+
|Final_Rating_Adjust|count(session_id)|
+-------------------+-----------------+
|                  1|         32930593|
|                  6|       1153802824|
|                  3|          8332379|
|                  5|          7127384|
|                  9|         87913811|
|                  4|           159804|
|                  8|         46665799|
|                  7|        510759745|
|                 10|          1727159|
|                 11|          4328875|
|                  2|        218254204|
+-------------------+-----------------+



## Encoding

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

In [17]:
df4.printSchema()

root
 |-- session_id: string (nullable = true)
 |-- track_id_clean: string (nullable = true)
 |-- Final_Rating_Adjust: integer (nullable = true)



In [18]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df4) 
            for column in ["session_id", "track_id_clean"] ]

pipeline = Pipeline(stages=indexers)
df_index = pipeline.fit(df4).transform(df4)

df_index.show()

Py4JJavaError: An error occurred while calling o286.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 8 tasks (1054.4 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2041)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2029)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2028)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2028)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:966)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler

In [ ]:
training, test = df_index.randomSplit([0.8, 0.2])

In [ ]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="session_id_index", itemCol="track_id_clean_index", ratingCol="Final_Rating_Adjust",
          coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
save(model, "s3://olihw4//spotify_full_version_model")

In [41]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Final_Rating_Adjust",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 10.181823908017078


In [42]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

In [ ]:
userRecs.select("recommendations").show(10)

In [49]:
userRecs.printSchema()

root
 |-- session_id_index: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- track_id_clean_index: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [ ]:
movieRecs.show(5)